In [ ]:
import reservoirpy as rpy
import pandas as pd
import numpy as np

# rpy.verbosity(0)  # no need to be too verbose here
rpy.set_seed(42)  # make everything reproducible !

In [ ]:
from reservoirpy.nodes import Reservoir
# Create the reservoir
reservoir = Reservoir(100, lr=0.5, sr=0.9)

In [ ]:
# Load the data
data = pd.read_csv('incidents_and_temperatures_per_day.csv', header=0)
# Drop nan values if present
data = data.dropna() if data.isnull().values.any() else data
# Retrieve the needed data columns
X = data[['temperature', 'humidite', 'nuages']]
Y = data[['nb_incidents']]

In [ ]:
# Prepare the data for the reservoir (needs to be a numpy array)
X = np.array(X)
Y = np.array(Y)

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
from reservoirpy.nodes import Ridge
ridge = Ridge(ridge=1e-7)
esn_model = reservoir >> ridge

In [ ]:
train_states = reservoir.run(X_train)

esn_model = esn_model.fit(X_train, Y_train, warmup=10)

In [ ]:
# Check if the method has initialized all nodes in the Model, and trained the Ridge readout
print(reservoir.is_initialized, ridge.is_initialized, ridge.fitted)

In [ ]:
# Predict 
Y_pred = esn_model.run(X_test)
Y_pred